In [ ]:
!pip install pandas matplotlib numpy scikit-learn seaborn scikit-fuzzy openpyxl

In [1]:
# Gerekli kütüphaneleri dahil et
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import skfuzzy as fuzz
from sklearn.metrics import silhouette_score, calinski_harabasz_score,davies_bouldin_score
import random

In [2]:
# XLS dosyasını oku
df = pd.read_excel('sku_data.xlsx')

In [3]:
# Veri hakkında genel bilgileri göster
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2279 entries, 0 to 2278
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2279 non-null   int64  
 1   Unitprice        2279 non-null   float64
 2   Expire date      2279 non-null   int64  
 3   Outbound number  2279 non-null   int64  
 4   Total outbound   2279 non-null   float64
 5   Pal grossweight  2279 non-null   float64
 6   Pal height       2279 non-null   float64
 7   Units per pal    2279 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 142.6 KB
None


In [4]:
# İlk 3 satırı göster
print(df.head(3))

   ID  Unitprice  Expire date  Outbound number  Total outbound  \
0   1      0.058          547                9          2441.0   
1   2      0.954          547                0             0.0   
2   3      2.385          547               12            23.0   

   Pal grossweight  Pal height  Units per pal  
0           105.60        1.56           1920  
1           207.68        1.00            384  
2           165.78        1.02            108  


In [5]:
# Veri setindeki sayısal sütunların istatistiksel özetini al
print(df.describe())

                ID    Unitprice  Expire date  Outbound number  Total outbound  \
count  2279.000000  2279.000000  2279.000000      2279.000000     2279.000000   
mean   1140.000000     4.269402   410.371654       235.976305      731.701053   
std     658.034953    14.449000   240.875419       700.230685     2146.029848   
min       1.000000     0.000000     0.000000         0.000000        0.000000   
25%     570.500000     0.000000   365.000000         0.000000        0.000000   
50%    1140.000000     1.293800   547.000000         1.000000        3.000000   
75%    1709.500000     4.545000   547.000000        45.000000      419.500000   
max    2279.000000   518.592000   734.000000      6325.000000    26411.000000   

       Pal grossweight   Pal height  Units per pal  
count      2279.000000  2279.000000    2279.000000  
mean        192.939582     0.672798     755.563405  
std         164.616813     0.552117    6278.437915  
min           0.000000     0.000000       0.000000  
25%  

In [6]:
# 'ID' niteliğini çıkar
df = df.drop('ID', axis=1)

In [7]:
# Bağımsız bir kopya oluştur
df_backup = df.copy()

In [8]:
# Sütun bazında normalize etme
scaler = MinMaxScaler()
df_nor = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# Normalize edilmiş veri hakkında genel bilgileri göster
print(df_nor.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2279 entries, 0 to 2278
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unitprice        2279 non-null   float64
 1   Expire date      2279 non-null   float64
 2   Outbound number  2279 non-null   float64
 3   Total outbound   2279 non-null   float64
 4   Pal grossweight  2279 non-null   float64
 5   Pal height       2279 non-null   float64
 6   Units per pal    2279 non-null   float64
dtypes: float64(7)
memory usage: 124.8 KB
None


In [9]:
# Bağımsız bir kopya oluştur
df_backup_nor = df_nor.copy()

In [10]:
df = df_nor

In [11]:
# Verileri numpy dizisine dönüştürme
veri = df.values

# Küme sayıları aralığı
k_range = range(2, 21)

# İndeks değerlerini saklamak için boş listeler
silhouette_skorlari = []
ch_skorlari = []
db_skorlari = []

# Her bir küme sayısı için indeks hesaplama ve grafik oluşturma
for i, k in enumerate(k_range):
    # Fuzzy C-means uygulama
    random.seed(10)
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(veri.T, c=k, m=2, error=0.005, maxiter=1000)

    # Her bir veri noktası için tahmin edilen küme indekslerini belirleme
    kume_uyelikleri = np.argmax(u, axis=0)

    # Silhouette indeksini hesapla
    silhouette_ort = silhouette_score(veri, kume_uyelikleri)
    silhouette_skorlari.append(silhouette_ort)

    # Calinski-Harabasz indeksini hesapla
    ch_skoru = calinski_harabasz_score(veri, kume_uyelikleri)
    ch_skorlari.append(ch_skoru)

    # Davies-Bouldin indeksini hesapla
    db_indeksi = davies_bouldin_score(veri, kume_uyelikleri)
    db_skorlari.append(db_indeksi)

    # Indeks değerlerini ekrana yazdır
    print("Silhouette Indeksi:", silhouette_ort)
    print("Calinski-Harabasz Indeksi:", ch_skoru)
    print("Davies-Bouldin Indeksi:", db_indeksi)
    print("------------------------------------", k)




Silhouette Indeksi: 0.49629093955048886
Calinski-Harabasz Indeksi: 1942.5943509044173
Davies-Bouldin Indeksi: 0.7931659491669967
------------------------------------ 2
Silhouette Indeksi: 0.5122684428770979
Calinski-Harabasz Indeksi: 2162.800619211801
Davies-Bouldin Indeksi: 0.7542144605344291
------------------------------------ 3
Silhouette Indeksi: 0.5411899069079208
Calinski-Harabasz Indeksi: 2012.1976546050846
Davies-Bouldin Indeksi: 0.7291079082962212
------------------------------------ 4
Silhouette Indeksi: 0.47268179304124047
Calinski-Harabasz Indeksi: 1846.6269962796307
Davies-Bouldin Indeksi: 0.9160824755992751
------------------------------------ 5
Silhouette Indeksi: 0.38763344193370597
Calinski-Harabasz Indeksi: 1514.891284251281
Davies-Bouldin Indeksi: 1.12598014705683
------------------------------------ 6
Silhouette Indeksi: 0.43740449493392447
Calinski-Harabasz Indeksi: 1734.0346545349082
Davies-Bouldin Indeksi: 0.9975372227125068
------------------------------------ 

In [15]:
# Verileri numpy dizisine dönüştürme
df2 = df.copy()
veri = df2.values

# Fuzzy C-means uygulama
random.seed(10)
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(veri.T, c=4, m=2, error=0.005, maxiter=1000)

# Her bir veri noktası için tahmin edilen küme indekslerini belirleme
kume_uyelikleri = np.argmax(u, axis=0)

# Sonuçları atama
df2['Cluster'] = kume_uyelikleri
print(df2)

      Unitprice  Expire date  Outbound number  Total outbound  \
0      0.000112     0.745232         0.001423        0.092424   
1      0.001840     0.745232         0.000000        0.000000   
2      0.004599     0.745232         0.001897        0.000871   
3      0.009834     0.745232         0.000000        0.000000   
4      0.000000     0.745232         0.000000        0.000000   
...         ...          ...              ...             ...   
2274   0.000000     0.000000         0.000000        0.000000   
2275   0.000000     0.000000         0.000158        0.000038   
2276   0.000000     0.000000         0.000000        0.000000   
2277   0.000000     0.000000         0.000158        0.000038   
2278   0.000000     0.745232         0.012016        0.059445   

      Pal grossweight  Pal height  Units per pal  Cluster  
0            0.116402    0.722222       0.009600        2  
1            0.228924    0.462963       0.001920        2  
2            0.182738    0.472222      